In [1072]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats

In [1073]:
import random
random.seed(13)

## Import data

### Import data

In [1074]:
# delete
# code = pd.read_csv('data/coding7128.tsv', sep='\t')

In [998]:
# need to import this file in slightly different way because of different delimiters
clinical = pd.read_csv('data/covid19_clinical_sample.txt',sep='\t|,', engine = 'python')

In [999]:
ic10 = pd.read_csv('data/ic10_lookup.csv')

In [1000]:
GPic10 = pd.read_csv('data/GP_ic10_lookup.csv') 

In [1001]:
# need to import this file in slightly different way because of different delimiters
scripts = pd.read_csv('data/covid19_prescriptions_sample.txt', sep='\t|,', engine = 'python')

In [1002]:
mortality = pd.read_csv('data/patients_data_death_sample.csv')

In [1003]:
mortality_coding = pd.read_csv('data/patients_data_death_decoding.csv')

### Have a look at the data

In [1004]:
# code.head()
# CTV3 codes (used by GPs, in dataset clinical) and the corresponding disease and incidences
# code can also be measurements, but will not be used by us

In [1005]:
# delete all of these ouputs 

clinical
# GP dataset from every patient
# each line corresponds to one incident with a date (event_dt)
# eid: patient number 
# code_type: ?
# value: measured value of a test or whatever

,Unnamed: 0,eid,event_dt,code_type,code,value
0,0,1000034,25/07/2017,0,XaIyt,0.000000
1,1,1000034,26/06/2018,0,XaF8d,0.000000
2,2,1000034,22/08/2017,0,XaF8d,0.000000
3,3,1000034,04/08/2015,0,XaF8d,0.000000
4,4,1000034,07/06/2018,0,XaIeG,0.000000
...,...,...,...,...,...,...
999995,999995,1033328,23/05/2012,0,42N..,0.330000
999996,999996,1033328,10/07/2013,0,246A.,80.000000
999997,999997,1033328,23/05/2012,0,XaK8y,83.300003
999998,999998,1033328,20/05/1997,0,246A.,80.000000


In [1006]:
ic10
# description of ICD20 codes

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17931,17931,U84.8,U848,DEFAULT,3.0,Resistance to other specified antimicrobial drug,,,,,,,
17932,17932,U84.9,U849,DEFAULT,3.0,Resistance to unspecified antimicrobial drugs,,,,,,,
17933,17933,U85,U85X,DEFAULT,3.0,Resistance to antineoplastic drugs,,,,,,,
17934,17934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1007]:
GPic10
#translates CTV3 codes (read_code) to ICD10 (icd10_code)

,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
0,0,123..,Z831,D,C,M,0.0,0.0
1,1,123..,Z830,A,C,M,0.0,0.0
2,2,1231.,Z831,G,C,M,0.0,0.0
3,3,1232.,Z831,G,C,M,0.0,0.0
4,4,1233.,Z831,G,C,M,0.0,0.0
...,...,...,...,...,...,...,...,...
116372,116372,ZVy..,Z769,D,C,C,0.0,0.0
116373,116373,ZVz..,Z769,D,C,C,0.0,0.0
116374,116374,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116375,116375,"Contains information from NHS Digital, licence...",NaN,NaN,NaN,NaN,NaN,NaN


In [1008]:
scripts
# all medication patients got from their GP, incl. the COVID-19 vaccine 
# needs to be complete if want to compare vaccinated vs. non-vaccinated, but seems like not all vaccinations registered here but just the ones done by the GP

,Unnamed: 0,eid,issue_date,dmd_code
0,0,1000034,05/08/2016,317310006
1,1,1000034,16/03/2015,317310006
2,2,1000034,22/05/2017,317310006
3,3,1000034,31/03/2016,317310006
4,4,1000034,23/01/2019,317310006
...,...,...,...,...
999995,999995,1069267,22/06/2015,347928005
999996,999996,1069267,26/07/1997,382311000001103
999997,999997,1069267,13/03/1997,374296008
999998,999998,1069267,15/07/1997,382311000001103


In [1009]:
# delete 

# get few individuals from large dataset -> may try to call only part of scripts dataset, where code == one of covid vaccine

# try only to get lines from scripts dataset where dmd_code== a vaccine 
# use function read_csv. only where irgendwas

# print('lines in samplescripts: ', len(scripts))
# print('number of subjects in samplescripts: ', len(scripts['eid'].value_counts()))

## Preparing data

### Adding column with IC10 categories to clinical dataset for simpler analysis later on

In [1010]:
# create a reduced dataframe for merging with the clinical dataframe
GPic10_red =  GPic10[['read_code', 'icd10_code']]
# rename to merge ont he ctv3 column with the clinical dataset
GPic10_red = GPic10_red.rename(columns = {'read_code':'code'})

In [1011]:
clinical_ic10 = pd.merge(clinical, GPic10_red, how='inner', on='code')

### Data Cleaning

In [1012]:
# drop unnecessary columns
clinical_ic10.drop(columns = ['code_type', 'value', 'Unnamed: 0'], inplace = True)

In [1013]:
# delete 

pd.isnull(clinical_ic10).sum()
# no NaN

eid           0
event_dt      0
code          0
icd10_code    0
dtype: int64

In [1014]:
# delete

pd.isnull(ic10).sum()

Unnamed: 0          0
ICD10_CODE          1
ALT_CODE            2
USAGE               2
USAGE_UK            2
DESCRIPTION         2
MODIFIER_4          2
MODIFIER_5          2
QUALIFIERS          2
GENDER_MASK         2
MIN_AGE             2
MAX_AGE             2
TREE_DESCRIPTION    2
dtype: int64

In [1015]:
# remove duplicate entries in clinical dataset 
# (for other datasets, it should not be a problem)
clinical_ic10.drop_duplicates(inplace = True) 

# remove NaN in selected column
ic10.dropna(subset=['DESCRIPTION'], inplace = True)

In [1016]:
# delete
pd.isnull(ic10).sum()

Unnamed: 0          0
ICD10_CODE          0
ALT_CODE            0
USAGE               0
USAGE_UK            0
DESCRIPTION         0
MODIFIER_4          0
MODIFIER_5          0
QUALIFIERS          0
GENDER_MASK         0
MIN_AGE             0
MAX_AGE             0
TREE_DESCRIPTION    0
dtype: int64

In [1017]:
# delete
clinical_ic10

,eid,event_dt,code,icd10_code
0,1000034,13/05/1998,XE1lT,S832
1,1011402,15/08/2003,XE1lT,S832
2,1011402,11/03/2004,XE1lT,S832
3,1011402,16/07/2004,XE1lT,S832
4,1012829,06/01/2015,XE1lT,S832
...,...,...,...,...
188488,1033266,23/06/2015,XaDbE,R568
188489,1033328,11/12/2018,X2060,I839
188490,1033328,11/12/2018,X2060,I83
188491,1033328,11/12/2018,X2060,O220


In [1018]:
# remove not considered clinical events in the clinical dataset (e.g. external factors)
# remove O-Z, but leave R as adverse event & Q as underlying disease.

# tuple of categories we do not consider as adverse event or underlying disease:
exclude = ('O','P','S','T','U','V','W','X','Y','Z')

# broadcasting the clinical dataset for all icd codes except those we exclude
clinical_ic10 = clinical_ic10[~clinical_ic10['icd10_code'].str.startswith(exclude)]
# clinical_ic10

### Rename columns of mortality dataframe

In [1019]:
# delete
mortality.head()

,Unnamed: 0,eid,31-0.0,34-0.0,52-0.0,53-0.0,21001-0.0,21022-0.0,40000-0.0,40001-0.0,40001-1.0,40007-0.0
0,334517,4345610,1.0,1940.0,2.0,2009-09-17,28.4091,69.0,NaN,NaN,NaN,NaN
1,297000,3970369,1.0,1949.0,12.0,2008-04-18,26.1106,58.0,NaN,NaN,NaN,NaN
2,260822,3608558,1.0,1943.0,2.0,2010-02-26,27.4614,67.0,NaN,NaN,NaN,NaN
3,245361,3453904,0.0,1957.0,8.0,2010-03-19,29.3199,52.0,NaN,NaN,NaN,NaN
4,25616,1256219,1.0,1959.0,7.0,2008-12-20,32.7859,49.0,NaN,NaN,NaN,NaN


In [1020]:
# delete 
mortality_coding

,Unnamed: 0,description
0,eid,PatientID
1,21022-0.0,AgeRecruit
2,53-0.0,DateRecruit
3,34-0.0,YearBirth
4,52-0.0,MonthBirth
5,40007-0.0,AgeDeath
6,40000-0.0,DateDeath
7,31-0.0,Sex
8,21001-0.0,BMI
9,40001-0.0,cause of death 1


In [1021]:
# make a dictionary out of the mortality_coding dataframe 
dict_columns = {}
for index, value in enumerate(mortality_coding['Unnamed: 0']):
  dict_columns[value] = mortality_coding.loc[index, "description"]

In [1022]:
mortality.rename(columns = dict_columns, inplace = True) # columns = a dictionary will automatically rename the columns

# delete
mortality['DateDeath'].isna().sum()
# mortality['DateDeath'].value_counts()


9339

In [1023]:
# delete
mortality

,Unnamed: 0,PatientID,Sex,YearBirth,MonthBirth,DateRecruit,BMI,AgeRecruit,DateDeath,cause of death 1,cause of death 2,AgeDeath
0,334517,4345610,1.0,1940.0,2.0,2009-09-17,28.4091,69.0,NaN,NaN,NaN,NaN
1,297000,3970369,1.0,1949.0,12.0,2008-04-18,26.1106,58.0,NaN,NaN,NaN,NaN
2,260822,3608558,1.0,1943.0,2.0,2010-02-26,27.4614,67.0,NaN,NaN,NaN,NaN
3,245361,3453904,0.0,1957.0,8.0,2010-03-19,29.3199,52.0,NaN,NaN,NaN,NaN
4,25616,1256219,1.0,1959.0,7.0,2008-12-20,32.7859,49.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,218679,3187034,0.0,1969.0,4.0,2009-10-19,26.1005,40.0,NaN,NaN,NaN,NaN
9996,282089,3821234,0.0,1962.0,5.0,2010-03-02,28.3801,47.0,NaN,NaN,NaN,NaN
9997,219163,3191883,1.0,1964.0,1.0,2008-09-12,13.9123,44.0,NaN,NaN,NaN,NaN
9998,55963,1559739,0.0,1946.0,10.0,2008-10-08,25.4259,61.0,NaN,NaN,NaN,NaN


### Filter data by vaccination status

The meaning of the SNOMED codes ('dmd_code' in the 'scripts' dataset, where medications that the patients got are listed) were found on: https://services.nhsbsa.nhs.uk/dmd-browser/amp/view/159421?ref=YW1wTmFtZT1jb3ZpZCZzZWFyY2hUeXBlPUFNUCZzaG93SW52YWxpZEl0ZW1zPWZhbHNl#ampCodes

Each vaccine has a specific code.

In [1024]:
# broadcasting to find individuals with AstraZEneca vaccine
mask_AZ = scripts['dmd_code']== 39114911000001105

# delete
scripts[mask_AZ].head()

,Unnamed: 0,eid,issue_date,dmd_code
261135,261135,1018525,21/03/2021,39114911000001105
597692,597692,1040468,30/01/2021,39114911000001105
970763,970763,1066390,29/01/2021,39114911000001105
970765,970765,1066390,20/03/2021,39114911000001105


In [1025]:
# getting IDs of individuals with any corona vaccine first (compare vaccinates vs. not)
# may look at effects of specific vaccines later
dict_vac = {39326911000001101: 'Moderna', 39230211000001104: 'Janssen', \
            39826711000001101:'Medicago', 39473011000001103: 'Baxter', \
            39114911000001105: 'AstraZeneca', 39115611000001103: 'Pfizer',\
            39373511000001104: 'Valneva'}

# delete print in the loop
for dmd_code in dict_vac:
    mask_vac = scripts['dmd_code']== dmd_code
    print(dict_vac[dmd_code], ':',len(scripts[mask_vac]))

Moderna : 0
Janssen : 0
Medicago : 0
Baxter : 0
AstraZeneca : 4
Pfizer : 2
Valneva : 0


In [1026]:
# mask for any vaccine
mask_vac = scripts['dmd_code'].isin(dict_vac)

# number of vaccinated people
print('Number of vaccinated people:', len(scripts[mask_vac]))

Number of vaccinated people: 6


In [1027]:
# finding eids of vaccinated people in the dataset
id_vac = scripts[mask_vac]['eid'] 

# delete
id_vac

19471     1000899
261135    1018525
466220    1031590
597692    1040468
970763    1066390
970765    1066390
Name: eid, dtype: int64

In [1028]:
# finding events of the vaccinated people, but only events before vaccination found 
# -> may need a more recent dataset
# need to separate clinical events before & after vaccine
mask_vac_clin = clinical_ic10['eid'].isin(id_vac)

clinical_ic10[mask_vac_clin]

,eid,event_dt,code,icd10_code
2099,1000899,29/07/2013,X75rv,M255
2100,1000899,29/07/2013,X75rv,A180D
2101,1000899,29/07/2013,X75rv,M011A
2104,1000899,06/02/2020,X75rv,M255
2105,1000899,06/02/2020,X75rv,A180D
...,...,...,...,...
184032,1018525,30/03/2020,K24..,N459
184033,1018525,30/03/2020,K24..,N45
184034,1018525,22/03/2012,J323z,K429
184035,1018525,14/03/1995,H03z.,J039


### Categorize event as before or after COVID-19 vaccination / randomly allocated date 

#### Make 'event_date' in 'clinical' a datetime type

In [1029]:
# delete
clinical_ic10.head(10) 

,eid,event_dt,code,icd10_code
12,1000034,01/01/1978,XE0e7,N419
13,1000034,01/01/1978,XE0e7,N41
14,1013867,10/01/2003,XE0e7,N419
15,1013867,10/01/2003,XE0e7,N41
16,1000034,23/01/2019,XE1FC,M543
17,1000034,23/01/2019,XE1FC,M544
18,1000034,23/01/2019,XE1FC,G570
19,1000034,23/01/2019,XE1FC,M511
20,1000034,23/01/2019,XE1FC,M511D
21,1000034,23/01/2019,XE1FC,G551A


In [1030]:
from datetime import datetime, timedelta

In [1031]:
# convert entire event_dt in datetime format
clinical_ic10['event_date_format'] = pd.to_datetime(clinical_ic10['event_dt'])
# clinical_ic10

C:\Users\renee\AppData\Local\Temp/ipykernel_4844/3575346504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical_ic10['event_date_format'] = pd.to_datetime(clinical_ic10['event_dt'])


In [1032]:
# delete 
# checking the date range
clinical_ic10.sort_values(by = 'event_date_format', ascending = False)

,eid,event_dt,code,icd10_code,event_date_format
173433,1014125,07/07/2037,X785s,Q613,2037-07-07
183184,1016438,12/03/2021,X2033,I272,2021-12-03
183185,1016438,12/03/2021,X2033,I270,2021-12-03
160773,1028939,12/03/2021,XE0r3,R198,2021-12-03
121248,1028594,12/03/2021,XaE5D,M858,2021-12-03
...,...,...,...,...,...
81833,1000624,01/01/1901,M12z0,L309,1901-01-01
186024,1023271,01/01/1901,XaIRZ,J303,1901-01-01
82616,1009699,01/01/1901,G2...,I10X,1901-01-01
188015,1031708,01/01/1901,A741.,B330,1901-01-01


#### Subset 'scripts' to only those patients (eid) that got the COVID-19 vaccine and drop 'Unnamed: 0' column

In [1033]:
scripts.drop(columns = ['Unnamed: 0'], inplace = True)
mask_scripts_vac = scripts['dmd_code'].isin(dict_vac)
scripts_vaccines_only = scripts[mask_scripts_vac] 

len(scripts[mask_scripts_vac]) # corresponds to finding above

6

In [1034]:
# delete
scripts_vaccines_only.head() # contains eid and date of vaccination

,eid,issue_date,dmd_code
19471,1000899,09/01/2021,39115611000001103
261135,1018525,21/03/2021,39114911000001105
466220,1031590,23/02/2021,39115611000001103
597692,1040468,30/01/2021,39114911000001105
970763,1066390,29/01/2021,39114911000001105


In [1035]:
# gives warning (same as before, should maybe be checked!)
scripts_vaccines_only['issue_date_format'] = pd.to_datetime(scripts_vaccines_only['issue_date'])

C:\Users\renee\AppData\Local\Temp/ipykernel_4844/2480841435.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scripts_vaccines_only['issue_date_format'] = pd.to_datetime(scripts_vaccines_only['issue_date'])


In [1036]:
# delete
scripts_vaccines_only

,eid,issue_date,dmd_code,issue_date_format
19471,1000899,09/01/2021,39115611000001103,2021-09-01
261135,1018525,21/03/2021,39114911000001105,2021-03-21
466220,1031590,23/02/2021,39115611000001103,2021-02-23
597692,1040468,30/01/2021,39114911000001105,2021-01-30
970763,1066390,29/01/2021,39114911000001105,2021-01-29
970765,1066390,20/03/2021,39114911000001105,2021-03-20


In [1037]:
clinical_ic10.head()
# length: 4156
# len(clinical_ic10['eid'].unique()) # some eid are in there multiple times (of course)

,eid,event_dt,code,icd10_code,event_date_format
12,1000034,01/01/1978,XE0e7,N419,1978-01-01
13,1000034,01/01/1978,XE0e7,N41,1978-01-01
14,1013867,10/01/2003,XE0e7,N419,2003-10-01
15,1013867,10/01/2003,XE0e7,N41,2003-10-01
16,1000034,23/01/2019,XE1FC,M543,2019-01-23


#### Add deaths to clinical_ic10 df so they can be treated as adverse events

In [1038]:
# subset mortality df to the eids with a deathdate that is not NaN
mortality_sub = mortality.dropna(subset=['DateDeath'])
# drop all the unnecessary columns (also drop age and sex, perhaps no time to analyse this), keep eid, deathdate, cause
mortality_sub.drop(columns = ['Unnamed: 0', 'Sex', 'YearBirth', 'MonthBirth', 'DateRecruit', 'BMI', 'AgeDeath'], inplace = True)
# rename column 'PatientID' to 'eid' and 'DateDeath' to 'issue_date_format'
mortality_sub.rename(columns = {'PatientID':'eid', 'DateDeath':'event_date_format'}, inplace = True) 

C:\Users\renee\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\renee\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [1039]:
# add column 'icd10_code' to mortality, filled with string 'death'
mortality_sub['icd10_code'] = pd.Series(["death" for x in range(len(mortality.index))])


C:\Users\renee\AppData\Local\Temp/ipykernel_4844/2821320636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mortality_sub['icd10_code'] = pd.Series(["death" for x in range(len(mortality.index))])


In [1040]:
# make 'DateDeath' datetime type
mortality_sub['event_date_format'] = pd.to_datetime(mortality_sub['event_date_format'])


C:\Users\renee\AppData\Local\Temp/ipykernel_4844/1819874372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mortality_sub['event_date_format'] = pd.to_datetime(mortality_sub['event_date_format'])


In [1041]:
mortality_sub

,eid,AgeRecruit,event_date_format,cause of death 1,cause of death 2,icd10_code
5,2717656,68.0,2015-06-05,C259,NaN,death
16,1546385,47.0,2015-06-28,C64,NaN,death
34,3694551,68.0,2013-07-27,K720,NaN,death
35,3396715,65.0,2013-03-26,J449,NaN,death
69,2692340,68.0,2015-10-09,I251,NaN,death
...,...,...,...,...,...,...
9943,1525180,46.0,2011-09-26,C509,NaN,death
9969,3152046,60.0,2019-08-06,G409,NaN,death
9979,2244275,57.0,2017-11-16,C221,NaN,death
9985,3652978,53.0,2018-08-24,G35,NaN,death


In [1042]:
# now join it to the clinical_vac
clinical_ic10= pd.concat([clinical_ic10, mortality_sub], join = 'outer')

In [1043]:
# delete
clinical_ic10

,eid,event_dt,code,icd10_code,event_date_format,AgeRecruit,cause of death 1,cause of death 2
12,1000034,01/01/1978,XE0e7,N419,1978-01-01,NaN,NaN,NaN
13,1000034,01/01/1978,XE0e7,N41,1978-01-01,NaN,NaN,NaN
14,1013867,10/01/2003,XE0e7,N419,2003-10-01,NaN,NaN,NaN
15,1013867,10/01/2003,XE0e7,N41,2003-10-01,NaN,NaN,NaN
16,1000034,23/01/2019,XE1FC,M543,2019-01-23,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
9943,1525180,NaN,NaN,death,2011-09-26,46.0,C509,NaN
9969,3152046,NaN,NaN,death,2019-08-06,60.0,G409,NaN
9979,2244275,NaN,NaN,death,2017-11-16,57.0,C221,NaN
9985,3652978,NaN,NaN,death,2018-08-24,53.0,G35,NaN


#### Add column with the date of vaccination to clinical_ic10 and drop event_dt and issue_date

In [1044]:
# merge clinical_ic10 and scripts_vaccines_only to add the issue_date (date of vaccine) to the df
clinical_vac = pd.merge(clinical_ic10, scripts_vaccines_only, how ='outer', on ='eid')

In [1045]:
# convert issue date to date format
# clinical_vac['issue_date_format'] = pd.to_datetime(clinical_vac['issue_date'])
# renée: I guess it's safer, since the dtype is 'datetime64' in the new column and 'object' in the old one:
# print(clinical_vac['issue_date']) 
# print(clinical_vac['issue_date_format'])

In [1046]:
# drop event_dt and issue_date
clinical_vac.drop(columns = ['event_dt', 'issue_date'], inplace = True)

In [1047]:
len(clinical_vac) 
# total amounts of events: vaccinations and medical events from clinical_ic10

133690

In [1048]:
clinical_vac['issue_date_format'].isna().sum()

133422

#### Add column in clinical_vac that says whether a patient is vaccinated or not

In [1049]:
mask_vaccinated = clinical_vac['eid'].isin(id_vac)
clinical_vac['vaccinated'] = mask_vaccinated

# check how many events of vaccinated people there are
clinical_vac['vaccinated'].value_counts()

# clinical_ic10['eid'].loc[clinical_ic10['vaccinated']==True]

False    133422
True        268
Name: vaccinated, dtype: int64

In [1050]:
# delete
clinical_vac 
# 133029 events
# rows with event_dt=NaN: vaccinated eids without medical events -> maybe rename it

,eid,code,icd10_code,event_date_format,AgeRecruit,cause of death 1,cause of death 2,dmd_code,issue_date_format,vaccinated
0,1000034,XE0e7,N419,1978-01-01,NaN,NaN,NaN,NaN,NaT,False
1,1000034,XE0e7,N41,1978-01-01,NaN,NaN,NaN,NaN,NaT,False
2,1000034,XE1FC,M543,2019-01-23,NaN,NaN,NaN,NaN,NaT,False
3,1000034,XE1FC,M544,2019-01-23,NaN,NaN,NaN,NaN,NaT,False
4,1000034,XE1FC,G570,2019-01-23,NaN,NaN,NaN,NaN,NaT,False
...,...,...,...,...,...,...,...,...,...,...
133685,3652978,NaN,death,2018-08-24,53.0,G35,NaN,NaN,NaT,False
133686,2197584,NaN,death,2015-07-12,55.0,G122,NaN,NaN,NaT,False
133687,1040468,NaN,NaN,NaT,NaN,NaN,NaN,3.911491e+16,2021-01-30,True
133688,1066390,NaN,NaN,NaT,NaN,NaN,NaN,3.911491e+16,2021-01-29,True


#### Add random issue date to events of unvaccinated patients to 'issue_date_format'  in clinical_vac 

In [1051]:
# make list from 'issue_date' in scripts_vaccines_only
issue_dates = scripts_vaccines_only['issue_date_format'].tolist()


In [1052]:
# 'eid' of unvaccinated people
id_nvac = clinical_vac[clinical_vac['vaccinated']==False]['eid'].unique()

# delete
len(id_nvac)

1895

In [1053]:
# create list of new dates
issue_dt_random = random.choices(issue_dates, k = len(id_nvac))

# delete
len(issue_dt_random)

1895

In [1054]:
# make dictionary with random issue dates and unvaccinated eid
dict_nvac = dict(zip(id_nvac,issue_dt_random))

# delete
len(dict_nvac)

1895

In [1055]:
# fill na with mapping from the dictionary
clinical_vac['issue_date_format'] = clinical_vac['issue_date_format'].fillna(clinical_vac['eid'].map(dict_nvac))

In [1056]:
# delete

clinical_vac['issue_date_format'][clinical_vac['vaccinated']==True].unique()

array(['2021-09-01T00:00:00.000000000', '2021-03-21T00:00:00.000000000',
       '2021-02-23T00:00:00.000000000', '2021-01-30T00:00:00.000000000',
       '2021-01-29T00:00:00.000000000', '2021-03-20T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [1057]:
# delete

clinical_vac['issue_date_format'][clinical_vac['vaccinated']==False].unique()

array(['2021-03-21T00:00:00.000000000', '2021-01-29T00:00:00.000000000',
       '2021-03-20T00:00:00.000000000', '2021-09-01T00:00:00.000000000',
       '2021-01-30T00:00:00.000000000', '2021-02-23T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [1058]:
# delete

clinical_vac[clinical_vac['vaccinated'] == False]

,eid,code,icd10_code,event_date_format,AgeRecruit,cause of death 1,cause of death 2,dmd_code,issue_date_format,vaccinated
0,1000034,XE0e7,N419,1978-01-01,NaN,NaN,NaN,NaN,2021-03-21,False
1,1000034,XE0e7,N41,1978-01-01,NaN,NaN,NaN,NaN,2021-03-21,False
2,1000034,XE1FC,M543,2019-01-23,NaN,NaN,NaN,NaN,2021-03-21,False
3,1000034,XE1FC,M544,2019-01-23,NaN,NaN,NaN,NaN,2021-03-21,False
4,1000034,XE1FC,G570,2019-01-23,NaN,NaN,NaN,NaN,2021-03-21,False
...,...,...,...,...,...,...,...,...,...,...
133682,1525180,NaN,death,2011-09-26,46.0,C509,NaN,NaN,2021-03-21,False
133683,3152046,NaN,death,2019-08-06,60.0,G409,NaN,NaN,2021-03-21,False
133684,2244275,NaN,death,2017-11-16,57.0,C221,NaN,NaN,2021-01-30,False
133685,3652978,NaN,death,2018-08-24,53.0,G35,NaN,NaN,2021-03-20,False


In [1059]:
# delete 
clinical_vac['issue_date_format'].isna().sum()

0

#### Add column where it says if the event (event_date_format) was before or after the vaccination (issue_date)

In [1060]:
before_after_vaccine = []
for i in range (len(clinical_vac)):
    if clinical_vac['event_date_format'].iloc[i] >= clinical_vac['issue_date_format'].iloc[i]:
        before_after_vaccine.append('after')
    else:
        before_after_vaccine.append('before')
clinical_vac['before_after_vaccine'] = before_after_vaccine

In [1061]:
# delete
clinical_vac

,eid,code,icd10_code,event_date_format,AgeRecruit,cause of death 1,cause of death 2,dmd_code,issue_date_format,vaccinated,before_after_vaccine
0,1000034,XE0e7,N419,1978-01-01,NaN,NaN,NaN,NaN,2021-03-21,False,before
1,1000034,XE0e7,N41,1978-01-01,NaN,NaN,NaN,NaN,2021-03-21,False,before
2,1000034,XE1FC,M543,2019-01-23,NaN,NaN,NaN,NaN,2021-03-21,False,before
3,1000034,XE1FC,M544,2019-01-23,NaN,NaN,NaN,NaN,2021-03-21,False,before
4,1000034,XE1FC,G570,2019-01-23,NaN,NaN,NaN,NaN,2021-03-21,False,before
...,...,...,...,...,...,...,...,...,...,...,...
133685,3652978,NaN,death,2018-08-24,53.0,G35,NaN,NaN,2021-03-20,False,before
133686,2197584,NaN,death,2015-07-12,55.0,G122,NaN,NaN,2021-09-01,False,before
133687,1040468,NaN,NaN,NaT,NaN,NaN,NaN,3.911491e+16,2021-01-30,True,before
133688,1066390,NaN,NaN,NaT,NaN,NaN,NaN,3.911491e+16,2021-01-29,True,before


In [1062]:
# delete

# checking how many events before and after
clinical_vac['before_after_vaccine'].value_counts()

# if event_date_format is NaT, is says before in 'before_after_vaccine'

before    133146
after        544
Name: before_after_vaccine, dtype: int64

## Analysing clinical events

### Delete (?)

IC10 codes are structured. The following website was used for looking up the meanings of the categories (e.g. respiratory diseases):
https://www.icd10data.com/ICD10CM/Codes

In [1063]:
# delete
ic10.head()

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,


the GP_ic10 has the read code which is the CTV3. 
(However, when things get nasty, there are multiple ICD10 codes per CTV3 codes and viceversa… there is a mapping status which tells you whether the code is primary, duplicated, to revision, etc)

In [1064]:
# WILL PROBABLY NOT USE THIS

# just to see if corresponding CTV3 to IC10 works

# broadcasting for vaccinated comorbidities(CTV3) in ICD10
#mask_vac_ic10 = GPic10['read_code'].isin(clinical_ic10[mask_vac_clin]['code']) 
#GPic10[mask_vac_ic10]

# mask_vac: people that got any vaccine
# search for patients that got the vaccine in GPic10
# and look what clinical events they had via the read_code (e.g. XE01qn)


if time: analyse specific diseases such as myocarditis etc.

In [1065]:
clinical_ic10

,eid,event_dt,code,icd10_code,event_date_format,AgeRecruit,cause of death 1,cause of death 2
12,1000034,01/01/1978,XE0e7,N419,1978-01-01,NaN,NaN,NaN
13,1000034,01/01/1978,XE0e7,N41,1978-01-01,NaN,NaN,NaN
14,1013867,10/01/2003,XE0e7,N419,2003-10-01,NaN,NaN,NaN
15,1013867,10/01/2003,XE0e7,N41,2003-10-01,NaN,NaN,NaN
16,1000034,23/01/2019,XE1FC,M543,2019-01-23,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
9943,1525180,NaN,NaN,death,2011-09-26,46.0,C509,NaN
9969,3152046,NaN,NaN,death,2019-08-06,60.0,G409,NaN
9979,2244275,NaN,NaN,death,2017-11-16,57.0,C221,NaN
9985,3652978,NaN,NaN,death,2018-08-24,53.0,G35,NaN


In [1066]:
ic10.head(3)

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,


In [1067]:
# mask for getting codes related to myocarditis
mask_myocarditis = ic10['DESCRIPTION'].str.contains('myocarditis', case = False)
ic10[mask_myocarditis]

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
3700,3700,I01.2,I012,DEFAULT,3.0,Acute rheumatic myocarditis,,,,,,,
3732,3732,I09.0,I090,DEFAULT,3.0,Rheumatic myocarditis,,,,,,,
3861,3861,I40,I40,DEFAULT,3.0,Acute myocarditis,,,,,,,
3862,3862,I40.0,I400,DEFAULT,3.0,Infective myocarditis,,,,,,,
3863,3863,I40.1,I401,DEFAULT,3.0,Isolated myocarditis,,,,,,,
3864,3864,I40.8,I408,DEFAULT,3.0,Other acute myocarditis,,,,,,,
3865,3865,I40.9,I409,DEFAULT,3.0,"Acute myocarditis, unspecified",,,,,,,
3866,3866,I41,I41,ASTERIS,2.0,Myocarditis in diseases classified elsewhere,,,,,,,
3867,3867,I41.0,I410,ASTERIS,2.0,Myocarditis in bacterial diseases classified e...,,,,,,,
3868,3868,I41.1,I411,ASTERIS,2.0,Myocarditis in viral diseases classified elsew...,,,,,,,


In [1068]:
# getting data with myocarditis

# the alternative code 'ALT_CODE' used here, as this was used in the lookup table GPic10
myocarditis = ic10[mask_myocarditis]['ALT_CODE']

clinical_ic10[clinical_ic10['icd10_code'].isin(myocarditis)]
# nobody had myocarditis in this restricted data

,eid,event_dt,code,icd10_code,event_date_format,AgeRecruit,cause of death 1,cause of death 2


In [1069]:
## ishaemic heart disease
mask_ihd = ic10['DESCRIPTION'].str.contains('ischaemic heart disease', case = False)
ischaemicHD = ic10[mask_ihd]['ALT_CODE']
clinical_ic10[clinical_ic10['icd10_code'].isin(ischaemicHD)]

,eid,event_dt,code,icd10_code,event_date_format,AgeRecruit,cause of death 1,cause of death 2
56090,1000118,07/02/2014,XaINF,I249,2014-07-02,NaN,NaN,NaN
56091,1000118,15/12/2013,XaINF,I249,2013-12-15,NaN,NaN,NaN
56092,1000118,14/10/2014,XaINF,I249,2014-10-14,NaN,NaN,NaN
56093,1000912,31/10/2008,XaINF,I249,2008-10-31,NaN,NaN,NaN
56094,1011364,05/03/2015,XaINF,I249,2015-05-03,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
180660,1013003,25/11/2013,G3y..,I258,2013-11-25,NaN,NaN,NaN
182750,1016004,02/09/2010,XaG1Q,I259,2010-02-09,NaN,NaN,NaN
182751,1016004,02/09/2010,XaG1Q,I25,2010-02-09,NaN,NaN,NaN
184395,1019090,18/01/1994,G20..,I24,1994-01-18,NaN,NaN,NaN


several comorbidities

In [1070]:
# getting people with specific diseases

# new column for filling with selected disease
clinical_ic10['selected disease'] = np.nan 

# list of specific diseases, to be decided on
diseases_selec = ['ischaemic heart disease', 'cardiomyopathy', 'heart failure',\
                  'myocarditis'] 

for disease in diseases_selec:
    
    # broadcasting for selected disease
    mask = ic10['DESCRIPTION'].str.contains(disease, case = False) 
    # broadcasting for selected disease
    sub_disease = clinical_ic10[clinical_ic10['icd10_code'].isin(ic10[mask]['ALT_CODE'])]
    
    for i in sub_disease.index:
        # fill dataset with selected disease label
        clinical_ic10.loc[i,'selected disease'] = disease 

In [1071]:
# counting how many people in the dataset have a specific disease
clinical_ic10['selected disease'].value_counts()

ischaemic heart disease    680
heart failure              188
cardiomyopathy              57
Name: selected disease, dtype: int64

## 1) Effect of the vaccination on adverse events

a) Comparison of occurence of adverse events after vaccination of vaccinated and unvaccinated people

b) Same comparison in subset of subjects with underlying medical condition

c) Same comparison in subset of subjects with specific underlying medical condition 

### 1a) Comparison of occurence of adverse events after vaccination of vaccinated and unvaccinated people

In [1075]:
# initialize column & index names
dict_vacORnot = {'Vaccinated':1, 'Not vaccinated':0} #vaccination status
ls_AE = ['AE', 'no AE'] # adverse events

#### Total number of adverse events 
Adverse events = medical events with ICD10 code after the (artificial) vaccination date

Total number of adverse events: (multiple per patient in some cases) vaccinated vs. non-vaccinated

In [1076]:
def incidence(df_event, id_vac, id_nvac):
    """
    Calculates the number of AEs per thousand individuals in a given population.
    
    Input:
    - df_event: a dataframe containing all events considered in the analysis (e.g. AEs)
    - id_vac: series containing ID's of each individual (with each individual listed once) within the given population. // only vaccinated!
    - id_nvac: same as id_vac but with all non-vaccinated individuals
    
    
    """
    
    # initialize df for putting in the values
    df_sum = pd.DataFrame(columns = ['Number of AEs', 'Number of individuals', 'Number of AEs per thousand individuals'], index = dict_vacORnot)
    
    for v in dict_vacORnot: # loop through vaccinated vs. not
        
        # put in the number of events (AEs) by subsetting the df_event into vaccinated/not vaccinated
        df_sum['Number of AEs'][v] = len(df_event[df_event['vaccinated'] == dict_vacORnot[v]])
    
        # if we have no event (happens in the subset), add 0.5 for testing the plot and analysis further below
        # -> still visible in the output that it was added
        if df_sum['Number of AEs'][v]==0:
            df_sum['Number of AEs'][v]=0.5
    
    # put number of vaccinated/not vaccinated people in the population into the summary df
    df_sum['Number of individuals']['Vaccinated'] = len(id_vac) # vaccinated (row)
    df_sum['Number of individuals']['Not vaccinated'] = len(id_nvac) # not vaccinated (row)
    
    # calculate AEs/individual
    df_sum['Number of AEs per thousand individuals'] = df_sum['Number of AEs']/df_sum['Number of individuals']*1000
    
    return df_sum

In [1077]:
# Make a df with only AEs (defined medical events w. ICD10 code after the (artificial) vaccination date)
# reminder: leave R as adverse event & Q as underlying disease.

# subset of clinical dataset for adverse events (=after vaccine)
df_AEs = clinical_vac[clinical_vac['before_after_vaccine']== 'after']

# remove Q (congenital diseases), which are not considered as adverse events
df_AEs = df_AEs[~df_AEs['icd10_code'].str.startswith('Q')]

In [1078]:
# remove deaths, which are not considered adverse events
df_AEs = df_AEs[df_AEs['icd10_code'] != 'death'] 

In [1079]:
# get number of people of the chosen population
# list of all IDs from the clinical dataset
id_clin = clinical['eid'].unique()

# list of all IDs from the scripts dataset
id_scr = scripts['eid'].unique()

# concatenate both lists to get all IDs in one list
id_all = np.concatenate((id_clin, id_scr))

# leave only unique IDs in the list to get the number of IDs using len()
id_all = np.unique(id_all)

# get array with IDs without vaccine by deleting the IDs with vaccine out of the array with all IDs
id_nvac = np.delete(id_all, np.isin(id_all,id_vac))

In [1080]:
# use the function to create the summary table
df_sum_all = incidence(df_AEs, id_vac, id_nvac)
df_sum_all

,Number of AEs,Number of individuals,Number of AEs per thousand individuals
Vaccinated,0.5,6,83.333333
Not vaccinated,531,2617,202.904089


Plot how oftern AEs occur in vaccinated vs. not vaccinated people for all individuals in the dataset.

In [ ]:
plt.figure(figsize = (5,5))
plt.bar(df_sum_all.index, df_sum_all['Number of AEs per thousand individuals'])
plt.ylabel('Number of AEs per thousand individuals')
plt.title('Adverse events in the whole population of the dataset')

#### Incidence of AEs in all subjects
Further analysis: number of eids with AE (not total AEs)

Make contingency table (not using pd.crosstab to avoid another huge dataset with all individuals and because we need to count unique IDs)

In [ ]:
def contingency(df_event, id_vac, id_nvac, columns = ls_AE, index = dict_vacORnot):
    """
    Makes a 2x2 contingency table, which can be used for the chi^2 test and calculating the risk ratio.
    
    Input:
    - df_event: a dataframe containing all events considered in the analysis (e.g. AEs)
    - id_vac: series containing ID's of each individual (with each individual listed once) within the given population.
    - id_nvac: same as id_vac but with all non-vaccinated individuals
    
    """

    # initialize contingency table
    crosstab = pd.DataFrame(columns = columns, index = index)
    
    # fill contingency table with values in the subset

    for v in dict_vacORnot: # loop throug vaccinated vs. not

        
        # with AEs

        # number of nonvaccinated/vaccinated people with AEs = number of individuals with AEs
        crosstab.loc[v,'AE'] = len(df_event[df_event['vaccinated']==dict_vacORnot[v]]['eid'].unique())
        
        # DELETE THIS FOR FINAL ANALYSIS
        # if we have no event (happens in the subset), add 1 for testing the plot and analysis further below
        # -> still visible in the output that it was added
        if crosstab.loc[v,'AE']==0:
            crosstab.loc[v,'AE']=1

            
        # without AEs

        if v == 'Vaccinated': # for vaccinated people

            # total number of vaccinated individuals - those with AEs
            crosstab.loc[v,'no AE'] = len(id_vac) - crosstab.loc[v,'AE']

        else: # for not vaccinated people
            
            # using id_vac
            crosstab.loc[v,'no AE'] = len(id_nvac) - crosstab.loc[v,'AE']
            
    
    # adding margins to the contingency table (=total value as row and column)
    
    # vaccinated vs. not
    crosstab.loc['Total',:] = np.sum(crosstab, axis = 0)
    # event vs. no event
    crosstab.loc[:,'Total'] = np.sum(crosstab, axis = 1)

    return crosstab
    

In [ ]:
len(id_vac)

In [ ]:
len(id_nvac)

In [ ]:
crosstab_all = contingency(df_AEs, id_vac, id_nvac)
crosstab_all

#### Chi-squared test

In [ ]:
# define significance treshold
alpha = 0.05

# perform chi-squared test:
stat_1a, p_1a, dof_1a, expected_1a = sp.stats.chi2_contingency(crosstab_all.iloc[0:2,0:2], 1)

p_1a

#### Risk ratio 
(similar to Barda et al.)

In [ ]:
def riskratio(crosstab, z = 1.96):
    """
    Calculates the risk ratio by adding a row and a column with the total count to the contingency table.
    
    Returns the risk ratio with the confidence interval (CI) as a df and the adjusted contingency table:
    (RR, crosstab)
    
    Important: the input (crosstab) needs to have the following format:
    - 2x2
    - column 0 = events (e.g. AEs), column 1 = non-events
    - row 0 = intervention (e.g. vaccinated), column 1 = non-vaccinated
    """
    
    # calculating the risk ratio

    # risk of getting AE when vaccinated
    R_vac = crosstab.loc['Vaccinated','AE']/crosstab.loc['Vaccinated','Total']
    # risk of getting "AE" when not vaccinated
    R_nvac = crosstab.loc['Not vaccinated', 'AE']/crosstab.loc['Not vaccinated', 'Total']

    # risk ratio
    RR = R_vac / R_nvac
    
    # natural log of the RR     
    logRR = np.log(RR) 

    # standard error
    se = np.sqrt(crosstab.loc['Vaccinated','no AE']/ \
                 (crosstab.loc['Vaccinated','AE']*crosstab.loc['Vaccinated','Total']) + \
                crosstab.loc['Not vaccinated','no AE']/ \
                (crosstab.loc['Not vaccinated','AE']*crosstab.loc['Not vaccinated','Total']))

    # Assuming normal distribution, 95% lie around 1.96 standard deviations of the mean.
    # as 95% CI is common, the z-score (z) 1.96 is put as the default value.

    # 1-alpha confidence interval (CI) 
    CI_lower = np.exp(np.log(RR) - se * z) # lower bound
    CI_upper = np.exp(np.log(RR) + se * z) # upper bound    
    
    # put the result in a series
    RR_result = pd.Series(data = [RR, CI_lower, CI_upper], index = ['RR', 'CI_lower', 'CI_upper'])
    
    return RR_result

Note: se and CI calculated as described on https://en.wikipedia.org/wiki/Relative_risk. 

In [ ]:
RR_all = riskratio(crosstab_all)
RR_all

### 1b) Same comparison in subset of subjects with underlying medical condition


#### Subset df to eids with AEs and underlying disease

In [ ]:
# get dataset with only AEs AND people with underlying disease 
# = adverse events (events after vaccination) of people with underlying disease (= that had an event before vaccination)

# subset of the df with all AEs (df_AEs) with only people with underlying diseases (df_AEs_ud)
# reminder: leave R as adverse event (and general underlying disease) & Q as underlying disease.

# subset of clinical_vac df with only medical events before
df_ud= clinical_vac[clinical_vac['before_after_vaccine']== 'before']

# drop rows without icd10_code, which correspond to vaccinated people without underlying disease
df_ud.dropna(subset = ['icd10_code'], inplace = True)

# remove R (less well defined condition), which are not considered as underlying disease
# df_ud = df_ud[~df_ud['icd10_code'].str.startswith('R')]      # wieso uskommentiert?

# get id's of people with an underlying disease
id_ud = df_ud['eid'].unique()

# subset of the df with AEs with only people with an underlying disease
df_AEs_ud = df_AEs[df_AEs['eid'].isin(id_ud)]

In [ ]:
# individuals in subset with only people with underlying disease AND the vaccine
id_ud_vac = df_ud[df_ud['vaccinated']==True]['eid'].unique()

# individuals in subset with only people with underlying disease AND NOT the vaccine
id_ud_nvac = df_ud[df_ud['vaccinated']==False]['eid'].unique()

#### Incidence of AE in subjects with underlying disease

In [ ]:
df_sum_ud = incidence(df_AEs_ud, id_ud_vac, id_ud_nvac)
df_sum_ud

In [ ]:
plt.figure(figsize = (5,5))
plt.bar(df_sum_ud.index, df_sum_ud['Number of AEs per thousand individuals'])
plt.ylabel('Number of AEs per thousand individuals')
plt.title('Adverse events in subjects with underlying diseases')

#### Chi-squared test

In [ ]:
# contingency table
crosstab_ud = contingency(df_AEs_ud, id_ud_vac, id_ud_nvac, columns = ls_AE, index = dict_vacORnot)
crosstab_ud

In [ ]:
# define significance treshold
alpha = 0.05

# perform chi-squared test:
stat_2b, p_2b, dof_2b, expected_2b = sp.stats.chi2_contingency(crosstab_ud.iloc[0:2,0:2], 1)

p_2b

#### Risk ratio

In [ ]:
RR_ud = riskratio(crosstab_ud)
RR_ud

In [ ]:
# create a df for collecting the results of the risk ratio to compare the populations

# initialize
RR_result = pd.DataFrame(index = ['whole population', 'people with underlying diseases'], columns = ['RR', 'CI_lower', 'CI_upper'])

# fill it
# RR with data from all individuals in the scripts & clinical dataset
RR_result.loc['whole population',:] = RR_all
# RR with data from all individuals with underlying diseases
RR_result.loc['people with underlying diseases',:] = RR_ud

RR_result

In [ ]:
# plot the difference in RR with CI

# general figure configuration
plt.figure(figsize = (6,6))
plt.xscale("log")
plt.axvline(1, ls='--', linewidth=1, color='black')

# draw the RR
# RR results
x = RR_result['RR'].values
# error bar with same size in both directions
x_error = RR_result['RR']-RR_result['CI_lower']
# for equal spacing of the results
y = np.arange(len(RR_result))

# the plot
plt.errorbar(x, y, xerr = x_error, marker = "o", markersize = 10, color = 'b', ls='none')

# labelling
plt.xlabel('Risk Ratio (log scale)')
plt.title('Risk ratio of vaccination in people with underlying diseases or the general population') # renée: Risk ratio of vaccination on an adverse event in subjects with underlying diseases and the entire population
# labelling with ICD10 categories
plt.yticks(ticks = y, labels = RR_result.index) 
plt.ylim(-0.5,1.5)

### 1c) Same comparison in subset of subjects with specific underlying medical condition 

#### Loop through the ICD-10 categories

In [ ]:
# dictionary of disease categories to analyse, from https://www.icd10data.com/ICD10CM/Codes
dict_cat = {('A','B'):'Certain infectious and parasitic diseases',
           ('C','D0','D1','D2','D3','D4'):'Neoplasms',
           ('D5','D6','D7','D8'):'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism',
           'E':'Endocrine, nutritional and metabolic diseases',
           'F':'Mental, Behavioral and Neurodevelopmental disorders',
           'G':'Diseases of the nervous system',
           ('H0','H1','H2','H3','H4','H5'):'Diseases of the eye and adnexa',
           ('H6','H7','H8', 'H9'): 'Diseases of the ear and mastoid process',
           'I':'Diseases of the circulatory system',
           'J':'Diseases of the respiratory system',
           'K':'Diseases of the digestive system',
           'L':'Diseases of the skin and subcutaneous tissue',
           'M':'Diseases of the musculoskeletal system and connective tissue',
           'N':'Diseases of the genitourinary system',
           'Q':'Congenital malformations, deformations and chromosomal abnormalities'}

# As mentioned by Barda et al., adjustment for multiple comparisons is not commonly done in studies regarding safety.
alpha = 0.05

# initialize 
# dataframe for collecting number of AEs per thousands with/without vaccinated people for all categories
df_sum_cat_all = pd.DataFrame(columns = dict_vacORnot, index=dict_cat.values()) 

# df for saving categories & their p-values
#df_chi2_cat = pd.DataFrame(columns = ['p-value', 'significant'], index=dict_cat.values())

# df for saving the risk ratio and CIs for the plot
RR_result_cat = pd.DataFrame(index = dict_cat.values(), columns = ['RR', 'CI_lower', 'CI_upper'])

# df for saving categories and their p-value, significance, RR and CI
df_combined = pd.DataFrame(columns = ['P-value', 'Significant', 'Risk Ratio', 'Confidence Interval'], index = dict_cat.values())



for i, ic in enumerate(dict_cat): # loop over ICD10 codes of each disease category
    
    # get dataset with only AEs AND people with specific underlying disease
    # subset of the df with all AEs (df_AEs) with only people with specific underlying diseases
    # reminder: leave R as adverse event & Q as underlying disease.
    # df_ud = subset of clinical_vac df with only diagnoses before vaccine as defined further above

    # subset with only underlying diseases within the specific category
    df_cat = df_ud[df_ud['icd10_code'].str.startswith(ic)]

    # IDs with underlying disease in the specific category
    id_cat = df_cat['eid'].unique()

    # subset of the df with AEs with only people with underlying disease in the specific category
    df_AEs_cat = df_AEs[df_AEs['eid'].isin(id_cat)]

    # get IDs of individuals in this subset with vs without vaccine 
    id_cat_vac = df_cat[df_cat['vaccinated']==True]['eid'].unique() # with vaccine
    id_cat_nvac = df_cat[df_cat['vaccinated']==False]['eid'].unique() # without vaccine

    ################################################################################
    # calculate the number of AEs per thousand people and save the result in a table
    df_sum_cat = incidence(df_AEs_cat, id_cat_vac, id_cat_nvac)
    
    # put in the values from the summary incidence table
    df_sum_cat_all.loc[dict_cat[ic],:] = df_sum_cat.loc[:, 'Number of AEs per thousand individuals']
    
    
    ################################################################################    
    # contingency table
    crosstab_cat = contingency(df_AEs_cat, id_cat_vac, id_cat_nvac, columns = ls_AE, index = dict_vacORnot)
    
    ###############################################################################
    # perform chi-squared test:
    stat, p, dof, expected = sp.stats.chi2_contingency(crosstab_cat.iloc[0:2,0:2], 1)     ## changed crosstab_ud to crosstab_cat

    # save the p-value in the df
    df_combined.loc[dict_cat[ic],'P-value'] = p
    
    # mark in which categories there is a significant difference between vaccinated and not vaccinated
    if p <= alpha/2: # divided by two because of two-sided test (number of AEs could be higher or lower in vaccinated)
        df_combined.loc[dict_cat[ic],'Significant'] = 'Yes'
    else:
        df_combined.loc[dict_cat[ic],'Significant'] = 'No'
       
    
    ###############################################################################
    # risk ratio
    RR_cat = riskratio(crosstab_cat)

    # fill the df with the RR results for the plot
    RR_result_cat.loc[dict_cat[ic],:] = RR_cat
    
    CI_rounded = (round(RR_cat[1], 2), round(RR_cat[2], 2)) # round to one more decimal place than the original data has
    
    # newww fill df with RR
    df_combined.loc[dict_cat[ic], ['Risk Ratio', 'Confidence Interval']] = round(RR_cat[0],2) , CI_rounded
    
    


#### Total number of AEs in IC-10 categories

In [ ]:
# total number of AEs in different categories
df_sum_cat_all.sort_values(by = 'Vaccinated', ascending = False, inplace = True) # sorting
df_sum_cat_all

#### Outcomes of Chi squared test and RR

In [ ]:
# df with p-value, significance, RR and CI
df_combined

In [ ]:
plt.figure(figsize = (8,8))

# plot results for vaccinated and not on top of each other
for v in dict_vacORnot:
    plt.barh(df_sum_cat_all.index, df_sum_cat_all[v], alpha = 0.5, label = v)

# labelling
plt.title('Adverse events in people with diseases in specific ICD10 categories')
plt.xlabel('Number of AEs per thousand individuals')
plt.legend(loc = "upper right")


In [ ]:
# plot the RR with CI

# general figure configuration
plt.figure(figsize = (6,6))
plt.xscale("log")
plt.axvline(1, ls='--', linewidth=1, color='black')

# draw the RR
# RR results
x = RR_result_cat['RR'].values
# error bar with same size in both directions
x_error = RR_result_cat['RR']-RR_result_cat['CI_lower']
# for equal spacing of the results
y = np.arange(len(RR_result_cat))

# the plot
plt.errorbar(x, y, xerr = x_error, marker = "o", markersize = 10, color = 'b', ls='none')

# labelling
plt.xlabel('Risk Ratio (log scale)')
plt.title('Risk ratio of vaccination in people with underlying diseases in different ICD10 categories')
# labelling with ICD10 categories
plt.yticks(ticks = y, labels = RR_result_cat.index) 

## 2) Effect of the vaccination on mortality

a) Comparison of mortality rate after vaccination of vaccinated and unvaccinated people

b) Same comparison in subset of subjects with underlying medical condition

c) Same comparison in subset of subjects with specific underlying medical condition 


#### Incidence function to calculate mortality in 2c)

In [ ]:
# for c) specific underlying diseases
# df_event = deaths after vaccination in subjects with specific underlying diseases (before vaccine)
# id_vac = vaccinated subjects with specific underlying diseases
# id_nvac = unvaccinated subjects with specific underlying diseases

def incidence_deaths(df_event, id_vac, id_nvac):
    """
    Calculates the number of deaths per thousand individuals in a given population.
    
    Input:
    - df_event: a dataframe containing all events considered in the analysis (e.g. AEs)
    - id_vac: series containing vaccinated eids of the vaccinated subject of the given population
    - id_nvac: series containing non-vaccinated eids of the vaccinated subject of the given population
    
    
    """
    
    # initialize df for putting in the values
    df_sum = pd.DataFrame(columns = ['Number of deaths', 'Number of individuals', 'Number of deaths per thousand individuals'], index = dict_vacORnot)
    
    for v in dict_vacORnot: # loop through vaccinated vs. not
        
        # put in the number of events (AEs) by subsetting the df_event into vaccinated/not vaccinated
        df_sum['Number of deaths'][v] = len(df_event[df_event['vaccinated'] == dict_vacORnot[v]])
    
        # if we have no event (happens in the subset), add 0.5 for testing the plot and analysis further below
        # -> still visible in the output that it was added
        if df_sum['Number of deaths'][v]==0:
            df_sum['Number of deaths'][v]=0.5
    
    # put number of vaccinated/not vaccinated people in the population into the summary df
    df_sum['Number of individuals']['Vaccinated'] = len(id_vac) # vaccinated (row)
    df_sum['Number of individuals']['Not vaccinated'] = len(id_nvac) # not vaccinated (row)
    
    # calculate AEs/individual
    df_sum['Number of deaths per thousand individuals'] = df_sum['Number of deaths']/df_sum['Number of individuals']*1000
    
    return df_sum

### 2a) Comparison of mortality rate after vaccination of vaccinated and unvaccinated people

#### Subset clinical df to only death entries

In [ ]:
# subset of clinical_vac dataset for adverse events (=after vaccine)
df_deaths = clinical_vac[clinical_vac['before_after_vaccine']== 'after']

# make dataframe with only deaths 
df_deaths = df_deaths[df_deaths['icd10_code'] == 'death'] 

In [ ]:
# initialize column & index names
ls_death = ['Deaths', 'No deaths'] # deaths

#### Mortality in entire study population

Make contingency table (not using pd.crosstab to avoid another huge dataset with all individuals and because we need to count unique IDs)

In [ ]:
def contingency_deaths(df_event, id_vac, id_nvac, columns = ls_death, index = dict_vacORnot):
    """
    Makes a 2x2 contingency table, which can be used for the chi^2 test and calculating the risk ratio.
    
    Input:
    - df_event: a dataframe containing all events considered in the analysis (e.g. AEs)
    - id_vac: series, containing ID's of each individual (with each individual listed once) within the given population.
    - id_nvac: same as id_vac but with all non-vaccinated individuals
    
    """

    # initialize contingency table
    crosstab_deaths = pd.DataFrame(columns = columns, index = index)
    
    # fill contingency table with values in the subset

    for v in dict_vacORnot: # loop through vaccinated vs. not

        
        # with deaths

        # number of nonvaccinated/vaccinated people with deaths = number of individuals with deaths
        crosstab_deaths.loc[v,'Deaths'] = len(df_event[df_event['vaccinated']==dict_vacORnot[v]]['eid'].unique())
        
        # DELETE THIS FOR FINAL ANALYSIS
        # if we have no event (happens in the subset), add 1 for testing the plot and analysis further below
        # -> still visible in the output that it was added
        if crosstab_deaths.loc[v,'Deaths']==0:
            crosstab_deaths.loc[v,'Deaths']=1

            
        # without deaths

        if v == 'Vaccinated': # for vaccinated people

            # total number of vaccinated individuals - those with deaths
            crosstab_deaths.loc[v,'No deaths'] = len(id_vac) - crosstab_deaths.loc[v,'Deaths']
            
        else: # for not vaccinated people
            
            # total number of unvaccinated individuals - those with deaths
            crosstab_deaths.loc[v,'No deaths'] = len(id_nvac) - crosstab_deaths.loc[v,'Deaths']
            
    
    # adding margins to the contingency table (=total value as row and column)
    
    # vaccinated vs. not
    crosstab_deaths.loc['Total',:] = np.sum(crosstab_deaths, axis = 0)
    # event vs. no event
    crosstab_deaths.loc[:,'Total'] = np.sum(crosstab_deaths, axis = 1)

    return crosstab_deaths
    

In [ ]:
crosstab_all_deaths = contingency_deaths(df_deaths, id_vac, id_nvac)
crosstab_all_deaths

#### Chi-squared test

In [ ]:
# perform chi-squared test and name variavles differently for each test
# e.g. 1a, 1b, etc.
stat_2a, p_2a, dof_2a, expected_2a = sp.stats.chi2_contingency(crosstab_all_deaths.iloc[0:2,0:2], 1)

p_2a

#### Risk ratio

In [ ]:
def riskratio_deaths(crosstab, z = 1.96):
    """
    Calculates the risk ratio by adding a row and a column with the total count to the contingency table.
    
    Returns the risk ratio with the confidence interval (CI) as a df and the adjusted contingency table:
    (RR, crosstab)
    
    Important: the input (crosstab) needs to have the following format:
    - 2x2
    - column 0 = events (e.g. AEs), column 1 = non-events
    - row 0 = intervention (e.g. vaccinated), column 1 = non-vaccinated
    """
    
    # calculating the risk ratio

    # risk of getting AE when vaccinated
    R_vac = crosstab.loc['Vaccinated','Deaths']/crosstab.loc['Vaccinated','Total']
    # risk of getting "AE" when not vaccinated
    R_nvac = crosstab.loc['Not vaccinated', 'Deaths']/crosstab.loc['Not vaccinated', 'Total']

    # risk ratio
    RR = R_vac / R_nvac
    
    # natural log of the RR     
    logRR = np.log(RR) 

    # standard error
    se = np.sqrt(crosstab.loc['Vaccinated','No deaths']/ \
                 (crosstab.loc['Vaccinated','Deaths']*crosstab.loc['Vaccinated','Total']) + \
                crosstab.loc['Not vaccinated','No deaths']/ \
                (crosstab.loc['Not vaccinated','Deaths']*crosstab.loc['Not vaccinated','Total']))

    # Assuming normal distribution, 95% lie around 1.96 standard deviations of the mean.
    # as 95% CI is common, the z-score (z) 1.96 is put as the default value.

    # 1-alpha confidence interval (CI) 
    CI_lower = np.exp(np.log(RR) - se * z) # lower bound
    CI_upper = np.exp(np.log(RR) + se * z) # upper bound    
    
    # put the result in a series
    RR_result_deaths = pd.Series(data = [RR, CI_lower, CI_upper], index = ['RR', 'CI_lower', 'CI_upper'])
    
    return RR_result_deaths

note: se and CI calculated as described on https://en.wikipedia.org/wiki/Relative_risk. 

In [ ]:
RR_all_deaths = riskratio_deaths(crosstab_all_deaths)
RR_all_deaths

### 2b) Same comparison in subset of subjects with underlying medical condition

In [ ]:
# subset of the df with deaths with only people with an underlying disease - non in this dataset
df_deaths_ud = df_deaths[df_deaths['eid'].isin(id_ud)]
df_deaths_ud

In [ ]:
# should we keep this output?

# also here is total number of deaths not needed, as it is the same as later seen in the crosstable.
df_sum_ud = incidence_deaths(df_deaths_ud, id_ud_vac, id_ud_nvac)
df_sum_ud

In [ ]:
# delete?
plt.figure(figsize = (5,5))
plt.bar(df_sum_ud.index, df_sum_ud['Number of deaths per thousand individuals'])
plt.ylabel('Number of deaths per thousand individuals')
plt.title('Deaths in people with underlying diseases')

Chi-squared test

In [ ]:
# contingency table
crosstab_ud_deaths = contingency_deaths(df_deaths_ud, id_ud_vac, id_ud_nvac, columns = ls_death, index = dict_vacORnot)
crosstab_ud_deaths

In [ ]:
# perform chi-squared test:
stat_2b, p_2b, dof_2b, expected_2b = sp.stats.chi2_contingency(crosstab_ud_deaths.iloc[0:2,0:2], 1)

p_2b

Calculate and plot risk ratio

In [ ]:
RR_ud_deaths = riskratio_deaths(crosstab_ud_deaths)
RR_ud_deaths

In [ ]:
# create a df for collecting the results of the risk ratio to compare the populations

# initialize
RR_result_deaths = pd.DataFrame(index = ['Whole population', 'People with underlying diseases'], columns = ['RR', 'CI_lower', 'CI_upper'])

# fill it
# RR with data from all individuals in the scripts & clinical dataset
RR_result_deaths.loc['Whole population',:] = RR_all_deaths
# RR with data from all individuals with underlying diseases
RR_result_deaths.loc['People with underlying diseases',:] = RR_ud_deaths

RR_result_deaths

In [ ]:
# plot the difference in RR with CI

# general figure configuration
plt.figure(figsize = (6,6))
plt.xscale("log")
plt.axvline(1, ls='--', linewidth=1, color='black')

# draw the RR
# RR results
x = RR_result_deaths['RR'].values
# error bar with same size in both directions
x_error = RR_result_deaths['RR']-RR_result_deaths['CI_lower']
# for equal spacing of the results
y = np.arange(len(RR_result_deaths))

# the plot
plt.errorbar(x, y, xerr = x_error, marker = "o", markersize = 10, color = 'b', ls='none')

# labelling
plt.xlabel('Risk Ratio (log scale)')
plt.title('Risk ratio of vaccination on death in subjects with underlying diseases or the general population') 
# labelling with ICD10 categories
plt.yticks(ticks = y, labels = RR_result_deaths.index) 
plt.ylim(-0.5,1.5)

### 2c) Same comparison in subset of subjects with specific underlying medical condition 

In [ ]:
# initialize 

# dataframe for collecting number of AEs per thousands with/without vaccinated people for all categories
df_sum_cat_all_deaths = pd.DataFrame(columns = dict_vacORnot, index=dict_cat.values()) 

# df for saving the risk ratio and CI
RR_result_cat_deaths = pd.DataFrame(index = dict_cat.values(), columns = ['RR', 'CI_lower', 'CI_upper'])

# df for saving categories and their p-value, significance, RR and CI
df_combined_deaths = pd.DataFrame(columns = ['P-value', 'Significant', 'Risk Ratio', 'Confidence Interval'], index = dict_cat.values())


for i, ic in enumerate(dict_cat): # loop over ICD10 codes of each disease category
    
    # get dataset with only deaths AND people with specific underlying disease
    # subset of the df with all deaths (df_deaths) with only people with specific underlying diseases
    # reminder: leave R as adverse event & Q as underlying disease.
    # df_ud = subset of clinical_vac df with only diagnoses before vaccine as defined further above

    # subset with only underlying diseases within the specific category (=ic)
    df_cat = df_ud[df_ud['icd10_code'].str.startswith(ic)]

    # IDs with underlying disease in the specific category
    id_cat = df_cat['eid'].unique()

    # subset of the df with deaths after vaccination only in people with underlying disease in the specific category
    df_deaths_cat = df_deaths[df_deaths['eid'].isin(id_cat)]

    # get IDs of individuals in this subset with vs without vaccine 
    id_cat_vac = df_cat[df_cat['vaccinated']==True]['eid'].unique() # with vaccine
    id_cat_nvac = df_cat[df_cat['vaccinated']==False]['eid'].unique() # without vaccine

    ################################################################################
    # calculate the number of deaths per thousand people and save the result in a table
    df_sum_cat_deaths = incidence_deaths(df_deaths_cat, id_cat_vac, id_cat_nvac)
    
    # put in the values from the summary table
    df_sum_cat_all_deaths.loc[dict_cat[ic],:] = df_sum_cat_deaths.loc[:,'Number of deaths per thousand individuals']
    
    
    ################################################################################    
    # contingency table
    crosstab_cat = contingency_deaths(df_deaths_cat, id_cat_vac, id_cat_nvac, columns = ls_death, index = dict_vacORnot)
    
    ###############################################################################
    # perform chi-squared test:
    stat, p, dof, expected = sp.stats.chi2_contingency(crosstab_cat.iloc[0:2,0:2], 1)       # changed to crosstab_cat from crosstab_ud
    
    # save the p-value in the df
    df_combined_deaths.loc[dict_cat[ic],'P-value'] = p
    
    # mark in which categories there is a significant difference between vaccinated and not vaccinated
    if p <= alpha/2: # divided by two because of two-sided test (number of AEs could be higher or lower in vaccinated)
        df_combined_deaths.loc[dict_cat[ic],'Significant'] = 'Yes'
    else:
        df_combined_deaths.loc[dict_cat[ic],'Significant'] = 'No'
       
    ###############################################################################
    # risk ratio
    RR_cat = riskratio_deaths(crosstab_cat)

    # fill the df with the RR results for the plot later
    RR_result_cat_deaths.loc[dict_cat[ic],:] = RR_cat
    
    CI_rounded = (round(RR_cat[1], 2), round(RR_cat[2], 2)) # round to one more decimal place than the original data has
    
    # fill df with RR
    df_combined_deaths.loc[dict_cat[ic], ['Risk Ratio', 'Confidence Interval']] = round(RR_cat[0],2) , CI_rounded
    

df_sum_cat_all_deaths.sort_values(by = 'Vaccinated', ascending = False, inplace = True) # sorting
df_sum_cat_all_deaths

In [ ]:
# df with p-value, significance, RR and CI
df_combined_deaths

In [ ]:
plt.figure(figsize = (8,8))

# plot results for vaccinated and not on top of each other
for v in dict_vacORnot:
    plt.barh(df_sum_cat_all_deaths.index, df_sum_cat_all_deaths[v], alpha = 0.5, label = v)

# labelling
plt.title('Deaths in subjects with underlying diseases in specific ICD10 categories')
plt.xlabel('Number of deaths per thousand individuals')
plt.legend(loc = "upper right")


In [ ]:
# plot the RR with CI

# general figure configuration
plt.figure(figsize = (6,6))
plt.xscale("log")
plt.axvline(1, ls='--', linewidth=1, color='black')

# draw the RR
# RR results
x = RR_result_cat_deaths['RR'].values
# error bar with same size in both directions
x_error = RR_result_cat_deaths['RR']-RR_result_cat_deaths['CI_lower']
# for equal spacing of the results
y = np.arange(len(RR_result_cat_deaths))

# the plot
plt.errorbar(x, y, xerr = x_error, marker = "o", markersize = 10, color = 'b', ls='none')

# labelling
plt.xlabel('Risk Ratio (log scale)')
plt.title('Risk ratio of vaccination in people with underlying diseases in different ICD10 categories') # renée: Risk ratio of vaccination on an adverse event in subjects with underlying diseases in specific ICD-10 categories
# labelling with ICD10 categories
plt.yticks(ticks = y, labels = RR_result_cat_deaths.index) 

## Unused code so far

In [ ]:
# code layout for getting the different categories

# subsetting into categories using IC10
# e.g. I00-I99 Diseases of the circulatory system, more details available too, e.g. Ischemic heart diseases
GPic10.dropna(inplace=True) # drop NaN to enable making mask

m_circ = GPic10['icd10_code'].str.startswith('I')
GPic10[m_circ]['read_code']

In [ ]:
# getting corresponding CTV3 codes
m_circ_CTV3 = code['coding'].isin(GPic10[m_circ]['read_code'])
# code[m_circ_CTV3]

In [ ]:
 # getting subjects with this condition
m_circ_sub = clinical['code'].isin(code[m_circ_CTV3]['coding']) # another mask

### correspond to vaccinated or not

In [ ]:
# no one with any vaccine had circulatory issue before vaccine on this limited dataset
scripts[mask_vac]['eid'].isin(clinical[m_circ_sub]['eid']).sum()